In [2]:
import xml.etree.ElementTree as ET
import re
tree = ET.parse('data.xml')
root = tree.getroot()

In [3]:

citylist =dict(Berlin="BERLIN IS AWESOME")
STOPS = ["\{", "\}", "\[", "\]", "\\n","\\"]

counter = 0
key = ""
description = ""

word = 'fubar'
re_title = re.compile(r'title')
re_revision = re.compile(r'revision')
re_text = re.compile(r'text')
re_cutter = re.compile(r'[^=]*==')


for child in root:
    if (child.tag == "{http://www.mediawiki.org/xml/export-0.10/}siteinfo"):
        root.remove(child)
    else:
        #print 'CHILD'
        #print child.tag, child.attrib
        for subchild in child:
            if re_title.search(subchild.tag):
                #print "TITLEEEE"
                #print subchild.text
                key = subchild.text
            if re_revision.search(subchild.tag):
               # print 'REVISION', subchild.tag, subchild.attrib
                for text in subchild.findall('{http://www.mediawiki.org/xml/export-0.10/}text'):
                    if(len(text.text)>15):
                        description = text.text
                        for section in re_cutter.finditer(description):# Remove anything after first == appearance
                            description = section.group(0)
                            description = description[:-3]#Remove also the == that were included in the regex
                            break;
                        description = re.sub("\[\[[^\]]*\]", " ",description) # Remove everything between [[...]]
                        description = re.sub("\{\{[^\}]*\}", "", description) # Remove everything between {{...}}
                        description = re.sub("'''", "'", description) # Handle the ''''
                        description = re.sub("[\\\{\}\[\]\\n]","", description) # Remove stuff

                    
            citylist[key] = description
                    

        counter += 1
        if (counter >= 100):
            break;
    
    

In [64]:
#citylist.viewkeys()

In [4]:
abr = citylist["Abruzzo"]
abr

u" 'Abruzzo' is one of the twenty Italian regions. It is situated in the central part of  , to the east of Rome on the Adriatic Sea. Abruzzo's western border lies less than 50 miles due east of Rome. Abruzzo borders the region of Marche to the north, Lazio to the west and south-west, Molise to the south-east, and the Adriatic Sea to the east. Although geographically more of a central than southern region, ISTAT (the Italian statistical authority) considers it part of Southern Italy, a vestige of Abruzzo's historic association with the Kingdom of the Two Sicilies.Abruzzo is one of the most sparsely populated regions on the Italian peninsula. Always a wild and empty region, since the Second World War it has depopulated further, as people have left the land, and the traditional mainstay of sheep farming, for the cities and for America. A vacation in Abruzzo offers a taste of the unspoiled Italy."

# Seq2Vec own Model with Word2Vec average Vector

In [3]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import cPickle, gzip
import numpy as np
import pandas as pd


In [ ]:
EMBEDDING_FILE = './model/GoogleNews-vectors-negative300.bin'
#model = Word2Vec.load(EMBEDDING_FILE)

word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, \
        binary=True)
print "Loaded model"

In [9]:
word2vec

In [77]:
def vectorize(word2vec, sentence, num_features=300):
    mywords = mysentence.split(" ")
    myvector = np.zeros((num_features),dtype="float32")

    i = 0
    for word in mywords:
        #print word
        myvector = np.add(myvector,word2vec[word]) # Adding every new vector
        i+=1
    featureVec = np.divide(myvector, i) # and in the end dividing it by the number of words

    return featureVec

def eucDistance(vector1, vector2):
    euclid = 0.0
    for (dim1, dim2) in zip(vector1, vector2):
        euclid = euclid +(dim1-dim2)*(dim1-dim2)
        print dim1, dim2
    euclid = np.sqrt(euclid)
    return euclid
def maxDistance(vector1, vector2):
    return max(np.abs(np.substract(vector1,vector2)))
def minDistance(vec1, vec2):
    return min(np.abs(np.substract(vec1,vec2)))
def findBestMatch(vec1, vecarray):
    bestmatch = 1 # NEEDS TO BE DEFINED
    return bestmatch
def dfToVecArray(word2vec, df):
    vecarray = 1 # NNEDS TO BE DEFINED
    return vecarray

In [80]:
bsp1 = "Hi I am rather cool"
bsp2 = "Yo where have you been"
myvec1 = vectorize(word2vec, bsp1)
myvec2 = vectorize(word2vec, bsp2)
print [myvec1, myvec2]
euc = eucDistance(myvec1, myvec2)
euc

[array([  1.87011719e-01,   1.07519530e-01,   6.67480454e-02,
         1.15625001e-01,  -7.44628906e-02,   3.43261734e-02,
        -3.47412117e-02,  -5.39253242e-02,   9.29199234e-02,
         1.03561403e-02,   1.07421875e-02,  -1.66406244e-01,
        -9.81933624e-02,  -1.17968753e-01,  -2.07910150e-01,
         1.28955081e-01,  -1.58691406e-03,   1.07910156e-01,
         6.27868623e-02,  -1.08056642e-01,  -2.50579827e-02,
         8.99810791e-02,   1.18164062e-01,  -8.76464844e-02,
         4.09179702e-02,  -9.47265606e-03,  -6.39648456e-03,
        -1.40991213e-03,   6.92535415e-02,  -9.47265606e-03,
        -1.66992191e-02,   1.05761722e-01,  -4.21386734e-02,
        -7.61474594e-02,   4.15039062e-03,   7.86132812e-02,
         1.84277341e-01,   7.20214844e-03,   1.13671876e-01,
         2.12329105e-01,   1.23046879e-02,  -6.54052719e-02,
         2.10473627e-01,  -2.10876465e-02,   1.22070312e-02,
         9.86328162e-03,  -8.02123994e-02,  -7.24853501e-02,
         1.75292976e-02

0.0

In [10]:
def vectorize(model, words, num_features=300):
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0.
    index2word_set = set(model.wv.index2word)
    
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec, model[word])

    featureVec = np.divide(featureVec, nwords)
    return featureVec

In [ ]:

#model = Word2Vec.load("300features_40minwords_10context")
print "Loaded model"

df_train = "Hi over there I am the greatest"
print "Loaded train set"

generate(model, df_train, 'embeddings.npz')
print "Generated embeddings for train set"

In [ ]:
print('Processing text dataset')

# The function "text_to_wordlist" is from
# https://www.kaggle.com/currie32/quora-question-pairs/the-importance-of-cleaning-text
def text_to_wordlist(text, remove_stopwords=False, stem_words=False):
    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)

texts_1 = [] 
texts_2 = []
labels = []
with codecs.open(TRAIN_DATA_FILE, encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    header = next(reader)
    for values in reader:
        texts_1.append(text_to_wordlist(values[3]))
        texts_2.append(text_to_wordlist(values[4]))
        labels.append(int(values[5]))
print('Found %s texts in train.csv' % len(texts_1))